Note: We have checked the labels manually, this is the notebook for further proof.

## Loading Data

In [12]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score, confusion_matrix, classification_report, f1_score

In [29]:
df = pd.read_csv('manual_verified_labels.csv')

df.head()

,url,domain,title,date,clean_text,label,label_reason,source,year_month,year,month,tokens,text_len,label_machine,label_human_verified
0,https://market.bisnis.com/read/20200629/235/12...,market.bisnis.com,"Harga Emas Semakin Dekati US$1.800, Level Tert...",2020-06-29,Harga emas berjangka semakin mendekati level U...,Neutral,Kenaikan harga emas adalah tentang aset invest...,bisnis,2020-06,2020,6,426,3000,Neutral,Neutral
1,https://market.bisnis.com/read/20200826/93/128...,market.bisnis.com,Mantap! Rupiah Menguat Saat Mayoritas Mata Uan...,2020-08-26,Nilai tukar rupiah terhadap dolar Amerika Seri...,Deflation,Penguatan nilai tukar rupiah membuat harga bar...,bisnis,2020-08,2020,8,299,2072,Deflation,Deflation
2,https://market.bisnis.com/read/20200831/94/128...,market.bisnis.com,"Selain Emas, Harga Minyak Juga Terbantu Lesuny...",2020-08-31,Harga minyak mengalami penguatan seiring denga...,Inflation,Kenaikan harga minyak merupakan pendorong utam...,bisnis,2020-08,2020,8,188,1307,Inflation,Inflation
3,https://market.bisnis.com/read/20210910/235/14...,market.bisnis.com,"Naik Tipis! Harga Emas 24 Karat di Pegadaian, ...",2021-09-10,Harga emas batangan 24 karat yang dijual di Pe...,Neutral,Kenaikan harga emas batangan adalah tentang as...,bisnis,2021-09,2021,9,208,1465,Neutral,Neutral
4,https://market.bisnis.com/read/20200926/235/12...,market.bisnis.com,"Dolar AS Dinilai Lebih Aman, Harga Emas Jatuh",2020-09-26,Harga emas berjangka jatuh pada akhir pekan se...,Neutral,Penurunan harga emas berjangka mencerminkan pe...,bisnis,2020-09,2020,9,285,1938,Neutral,Neutral


## Kappa Evaluation

In [30]:
df['label_machine'] = df['label_machine'].astype(str).str.strip()
df['label_human_verified'] = df['label_human_verified'].astype(str).str.strip()

# Build consistent category set
categories = sorted(list(set(df['label_machine']) | set(df['label_human_verified'])))
cat_to_int = {c:i for i,c in enumerate(categories)}
int_to_cat = {i:c for c,i in cat_to_int.items()}

df['machine_int'] = df['label_machine'].map(cat_to_int)
df['human_int'] = df['label_human_verified'].map(cat_to_int)

print("Categories:", categories)

Categories: ['Deflation', 'Inflation', 'Neutral']


In [40]:
human = df['human_int'].astype(int)
machine = df['machine_int'].astype(int)

print(cohen_kappa_score(human, machine))
print(f1_score(human, machine, average='weighted'))

0.9670739224982027
0.9798969110937644


The Cohen’s Kappa score of 0.967 indicates an almost perfect agreement between the human-verified labels and the Gemini-generated labels, demonstrating very high consistency and reliability in the annotations, while the weighted F1 score of 0.98 suggests that the model’s predictions are nearly identical to the results of human verification. So, this means that the Gemini-generated labels are safe to use without fear of giving false information. We will now try with a few samples.

## Samples

Here, all sampling will go through a final check with GPT 5.1 to determine which label we will go with.

In [55]:
df['agree'] = df['label_machine'] == df['label_human_verified']
disagreements = df[~df['agree']]

NUM_SAMPLES = 10

sample_disagreement = disagreements.sample(NUM_SAMPLES, random_state=42)
df_sample = sample_disagreement.copy().reset_index(drop=True)

In [56]:
df_sample.head()

,url,domain,title,date,clean_text,label,label_reason,source,year_month,year,month,tokens,text_len,label_machine,label_human_verified,machine_int,human_int,agree
0,https://market.bisnis.com/read/20250317/93/186...,market.bisnis.com,"Dolar AS Loyo, Rupiah Dibuka Menguat ke Level ...",2025-03-17,Nilai tukar rupiah terhadap dolar Amerika Seri...,Deflation,Penguatan rupiah terhadap dolar AS akan menuru...,bisnis,2025-03,2025,3,385,2657,Deflation,Inflation,0,1,False
1,https://market.bisnis.com/read/20191025/235/11...,market.bisnis.com,"Pantau Pergerakan Harga Emas Comex, Jumat (25/...",2019-10-25,Harga emas di bursa Comex terpantau berbalik k...,Deflation,Koreksi harga emas ke zona merah menunjukkan p...,bisnis,2019-10,2019,10,467,2962,Deflation,Inflation,0,1,False
2,http://bisnis.liputan6.com/read/3153471/ini-pe...,bisnis.liputan6.com,"Ini Pendorong Ekonomi RI Tumbuh 5,06 Persen pa...",2017-11-06,Badan Pusat Statistik (BPS) mengumumkan pertum...,Neutral,Berita ini melaporkan pertumbuhan ekonomi dan ...,liputan6,2017-11,2017,11,105,766,Neutral,Inflation,2,1,False
3,http://market.bisnis.com/read/20171010/94/6977...,market.bisnis.com,"Persediaan Karet China Masih Tinggi, Karet Dit...",2017-10-10,Harga karet kembali ditutup melemah di hari ke...,Deflation,Penurunan harga karet akibat tingginya persedi...,bisnis,2017-10,2017,10,203,1406,Deflation,Neutral,0,2,False
4,https://www.liputan6.com/bisnis/read/4961787/j...,liputan6.com,Jaga Daya Beli Masyarakat Jadi PR Pemerintah d...,2022-05-13,", Jakarta Kepala Badan Kebijakan Fiskal, Kemen...",Inflation,Pemerintah mengakui kenaikan inflasi sebagai t...,liputan6,2022-05,2022,5,330,2379,Inflation,Neutral,1,2,False


### First Sample

In [69]:
df_sample.loc[2, 'clean_text']

'Badan Pusat Statistik (BPS) mengumumkan pertumbuhan ekonomi sebesar 5,06 persen di kuartal III-2017. Penyokong atau sumber utama pertumbuhan ekonomi ini, antara lain ekspor, impor, dan Pembentukan Modal Tetap Bruto (PMTB) atau invetasi. Kepala BPS, Suhariyanto atau yang akrab disapa Kecuk mengungkapkan, seluruh komponen pengeluaran tumbuh positif di kuartal III (year on year/yoy), antara lain konsumsi rumah tangga, konsumsi Lembaga Non Profit Rumah Tangga (LNPRT), konsumsi pemerintah, PMTB, ekspor dan impor. "Konsumsi rumah tangga 4,93 persen, konsumsi LNPRT tumbuh 6,01 persen, konsumsi pemerintah 3,46 persen, PMTB sebesar 7,11 persen, ekspor dan impor masing-masing tumbuh signifikan 17,27 persen serta 15,09 persen," kata dia di Jakarta, Senin (6/11/2017).'

In [70]:
df_sample.loc[2, 'label_machine']

'Neutral'

In [71]:
df_sample.loc[2, 'label_human_verified']

'Inflation'

Final decision

Neutral | Berita hanya melaporkan pertumbuhan ekonomi dan komponen PDB tanpa memberikan indikasi langsung terhadap kenaikan atau penurunan harga konsumen (CPI).

### Second Sample

In [76]:
df_sample.loc[4, 'clean_text']

', Jakarta Kepala Badan Kebijakan Fiskal, Kementerian Keuangan, Febrio Kacaribu mengatakan tahun ini tantangan yang dihadapi pemerintah bukan lagi dampak pandemi Covid-19. Melainkan menjaga daya beli masyarakat di tengah kenaikan inflasi yang juga dialami berbagai negara dunia. "Kalau tahun 2020 dan 2021 tuiga berat kita jaga kondisi kesehatan masyarakat. Tahun 2022 ini kesehatan masyarakat relatif baik. Maka fokus APBN sekarang pada kesehatan daya beli masyarakat," kata Febrio dalam Taklimat Media, Tanya BKF: Mengoptimalkan Sumber Pertumbuhan Ekonomi ke Depan, Jakarta, Jumat (13/5/2022). Baca Juga Dalam hal ini strategi yang dilakukan tidaklah sama. Instrumen APBN kali ini akan menjaga daya beli masyarakat sebagai syok absorber. Advertisement Pada konteks ini Febrio mengatakan pemerintah akan memanfaatkan kenaikan harga komoditas. Sehingga pemerintah optimis pertumbuhan ekonomi akan terus berlanjut tahun ini di tengah ancaman kenaikan inflasi. "Kabar gembira dengan komoditi boom dinik

In [77]:
df_sample.loc[4, 'label_machine']

'Inflation'

In [78]:
df_sample.loc[4, 'label_human_verified']

'Neutral'

Final decision

Inflation | Berita menekankan bahwa pemerintah fokus menjaga daya beli karena adanya kenaikan inflasi dan pemantauan kenaikan harga secara rutin, menunjukkan tekanan inflasi yang nyata.

### Third Sample

In [83]:
df_sample.loc[6, 'clean_text']

'Pemerintah terus mengimbau para pemudik untuk menjalankan mudik aman pada Lebaran tahun ini. Salah satunya adalah memastikan kendaraan dalam kondisi prima, termasuk kecukupan bahan bakar selama perjalanan. Harga bahan bakar minyak (BBM) di berbagai stasiun pengisian bahan bakar umum (SPBU) seperti Pertamina, Shell, BP AKR, dan Vivo mengalami penyesuaian per 1 April 2023. Berdasarkan situs resmi MyPertamina, Kamis (20/4/2023), harga Pertamax Turbo (RON 98) turun dari Rp15.100 per liter menjadi Rp15.000 per liter. Lalu, Dexlite turun dari Rp14.950 per liter menjadi Rp14.250 per liter. Sementara itu, harga Pertamina Dex dibanderol sebesar Rp15.400 per liter, atau turun Rp450 dari sebelumnya Rp15.850 per liter. Harga BBM bersubsidi Pertalite dan Solar belum mengalami penyesuaian, sehingga harga berlaku Rp10.000 untuk Pertalite dan Rp6.800 untuk Solar. Tak hanya Pertamina, SPBU Vivo juga melakukan penyesuaian harga BBM. Berdasarkan pantauan Bisnis di SPBU Vivo di Jalan Raya Pasar Minggu, J

In [84]:
df_sample.loc[6, 'label_machine']

'Deflation'

In [85]:
df_sample.loc[6, 'label_human_verified']

'Neutral'

Final Decision

Deflation | Berita menunjukkan penurunan harga berbagai jenis BBM, yang umumnya menurunkan tekanan inflasi karena BBM adalah komponen biaya transportasi dan produksi yang berpengaruh besar pada CPI.

### Fourth Sample

In [86]:
df_sample.loc[7, 'clean_text']

'Emiten properti pengelola Mal Pekanbaru, PT Bima Sakti Pertiwi Tbk. (PAMG), memaparkan strategi untuk meningkatkan tingkat kunjungan mal dan kinerja keuangan. Direktur Utama PAMG Christopher Sumasto Tjia mengatakan pada 2023, PAMG mencatatkan volume kunjungan masyarakat di Mal Pekanbaru sebanyak 7,41 juta orang, meningkat dibandingkan tahun sebelumnya yang tercatat sebanyak 6,86 juta orang. Pencapaian ini cukup memuaskan di tengah pertumbuhan ekonomi Indonesia yang positif dan ketidakpastian ekonomi global. "Kami terus melakukan peningkatan kinerja, dengan berupaya menambah penyewa untuk membuka tenant atau penyewaan toko di dalam mal dan menambah pengunjung mal. Kami juga telah berupaya menggencarkan pemasaran dalam berbagai kegiatan ataupun media promosi," kata Christopher Sumasto Tjia dalam siaran pers, Kamis (30/5/2024). Menurutnya, pertumbuhan ekonomi nasional mendorong perseroan untuk bersikap optimis dalam meningkatkan kinerja di tahun-tahun berikutnya. Hal itu sejalan dengan p

In [87]:
df_sample.loc[7, 'label_machine']

'Neutral'

In [88]:
df_sample.loc[7, 'label_human_verified']

'Deflation'

Final decision

Neutral | Berita hanya membahas strategi bisnis dan peningkatan kunjungan mal tanpa memberikan indikasi perubahan harga barang/jasa atau tekanan inflasi terhadap CPI Indonesia.

### Fifth Sample

In [89]:
df_sample.loc[8, 'clean_text']

'Gotion Indonesia menjalin kerja sama dengan ECGO untuk mewujudkan 1 juta pengendara motor listrik untuk ojek online (Ojol) dalam 3 tahun ke depan. Pada kesepakatan tersebut, Gotion Indonesia juga berkomitmen memasok 2 juta unit baterai ke seluruh kendaraan produksi ECGO yang menyasar para pengendara ojol di Tanah Air. Selain itu, jika target tersebut tercapai maka akan mengurangi 14 juta ton emisi karbon setiap tahun dan membantu penghematan subsidi Rp5,4 milyar per tahun untuk BBM. Advertisement “Kolaborasi dengan ECGO ini merupakan momen penting bagi kami saat kami berupaya menanamkan keberlanjutan di sektor-sektor ekonomi utama. Solusi baterai inovatif kami bukan hanya tentang memajukan teknologi tetapi sangat penting untuk membuat dampak nyata bagi tujuan Energi Nasional Indonesia, terutama dalam meningkatkan kualitas udara perkotaan dan meningkatkan kualitas hidup jutaan orang,” kata Anthony, Chairman Gotion Indonesia. Kerja sama 2 juta unit paket baterai ini akan dimulai pada ta

In [90]:
df_sample.loc[8, 'label_machine']

'Neutral'

In [91]:
df_sample.loc[8, 'label_human_verified']

'Inflation'

Final Decision

Neutral | Berita membahas kerja sama dan pengembangan ekosistem motor listrik tanpa menyinggung perubahan harga barang/jasa yang memengaruhi CPI Indonesia.